---
# Imports

In [1]:
# Write your code here. Add as many boxes as you need.
import pandas as pd
import numpy as np

---
# Laboratory Exercise - Run Mode (8 points)

## Introduction
In this laboratory assignment, the focus is on time series forecasting, specifically targeting the prediction of **atmospheric data** in Skopje. Your task involves employing bagging and boosting methods to forecast the **required measurements**. To accomplish this you will be using Pulse Eco Skopje measurements from **1 sensor**, which consists of the following values:

- Measurement Type (humidity, noise_dba, pm10, pm25, temperature)
- Value (Measured value for the given type)
- Stamp (A timestamp of the measurement)

**Note: You are required to perform this laboratory assignment on your local machine.**


# Read the data

You are required to load the given data from the `pulse_eco_one_sensor_data.csv` file. Load the data into a `pandas dataframe` and display the first 5 rows of the dataframe.

In [2]:
# Write your code here. Add as many boxes as you need.
df = pd.read_csv('pulse_eco_one_sensor_data.csv')
df.head()

,Type,Value,Stamp
0,humidity,87,2023-01-01T01:06:45+01:00
1,humidity,87,2023-01-01T01:22:18+01:00
2,humidity,87,2023-01-01T01:37:50+01:00
3,humidity,87,2023-01-01T01:53:21+01:00
4,humidity,87,2023-01-01T02:08:53+01:00


---
# Data Preprocessing

## Group the 'Stamp' column into 15 minute intervals

As the sensor produce data in infrequent intervals, we need to resample the data to 15 minute intervals. To do this you will need to:

- Convert the 'timestamp' column to a datetime object (use `pd.to_datetime` function ([Documentation](https://pandas.pydata.org/docs/reference/api/pandas.to_datetime.html)), with the `utc` argument set to `True`) 
- Floor each timestamp to the nearest 15 minutes (use the `.dt.floor` method ([Documentation](https://pandas.pydata.org/docs/reference/api/pandas.Series.dt.floor.html)))


Example of `.dt.floor` for 1h:
- data['TimeStamp'] = data['TimeStamp'].dt.floor('1h') 

In [3]:
df['Stamp'] = pd.to_datetime(df['Stamp'], utc=True)

In [4]:
# Write your code here. Add as many boxes as you need.
df['Stamp'] = df['Stamp'].dt.floor('15min')

Check if the operation was successful by displaying the first 5 rows of the data frame.

In [5]:
# Write your code here. Add as many boxes as you need.
df.head()

,Type,Value,Stamp
0,humidity,87,2023-01-01 00:00:00+00:00
1,humidity,87,2023-01-01 00:15:00+00:00
2,humidity,87,2023-01-01 00:30:00+00:00
3,humidity,87,2023-01-01 00:45:00+00:00
4,humidity,87,2023-01-01 01:00:00+00:00


## Sensor data to Pivot Table

To combine each measurement with the corresponding weather data, we will create a pivot table with the weather data and group the measurements by timestamp. Moreover, we will resample for each 15min, to add missing measurements.

**DO NOT MODIFY THE FOLLOWING CELLS, except for changing the df name.**

This operations will be given by the following code:

In [6]:
pivot = df.pivot_table(index='Stamp', columns='Type', values='Value', aggfunc='mean')
pivot = pivot.resample('15min').first()

In [7]:
pivot.head(3)

Type,humidity,noise_dba,pm10,pm25,temperature
Stamp,,,,,
2023-01-01 00:00:00+00:00,87.0,45.0,473.0,156.0,5.0
2023-01-01 00:15:00+00:00,87.0,44.0,333.0,120.0,5.0
2023-01-01 00:30:00+00:00,87.0,46.0,328.0,108.0,5.0


## Missing values

### Check for missing values 

Check if there are any missing values in the pivot data.

In [8]:
# Write your code here. Add as many boxes as you need.
pivot.isnull().sum()

Type
humidity       5010
noise_dba      3804
pm10           5317
pm25           5312
temperature    5010
dtype: int64

### Check for reason of missing values

Check if there is any reason for the missing data in the dataset.

In [9]:
# Write your code here. Add as many boxes as you need.

### Deal with missing values (Interpolation)

Use the `Interpolation` method to fill in the missing values in the dataset.

In [10]:
# Write your code here. Add as many boxes as you need.
pivot.interpolate(method='time', inplace=True)

#### Check if there are any missing values (Sanity Check)

Recheck for missing values.

In [11]:
# Write your code here. Add as many boxes as you need.
pivot.isnull().sum()

Type
humidity       0
noise_dba      0
pm10           0
pm25           0
temperature    0
dtype: int64

---
# Feature Engineering

## Drop Noise DBA as it is not relevant.

In [12]:
# Write your code here. Add as many boxes as you need.
pivot.drop(columns=['noise_dba'],inplace=True)

Check that the drop was successful.

In [13]:
# Write your code here. Add as many boxes as you need.
pivot.head()

Type,humidity,pm10,pm25,temperature
Stamp,,,,
2023-01-01 00:00:00+00:00,87.0,473.0,156.0,5.0
2023-01-01 00:15:00+00:00,87.0,333.0,120.0,5.0
2023-01-01 00:30:00+00:00,87.0,328.0,108.0,5.0
2023-01-01 00:45:00+00:00,87.0,415.0,125.0,5.0
2023-01-01 01:00:00+00:00,87.0,341.0,110.0,5.0


## Create Lag Features (3 prev values) for each measurement

Apply a lag of 15min, 30min, and 45min (3x15min intervals) to each feature, creating a set of features representing the meteorological conditions from the previous 45 minutes. To maintain dataset integrity, eliminate any resulting missing values at the beginning of the dataset.

Hint: Use `df['column_name'].shift(period)`. Check the documentation at https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.shift.html.

In [14]:
# Write your code here. Add as many boxes as you need.
for lag in range(1,4):
    for col in ['humidity',	'pm10',	'pm25',	'temperature']:
        pivot[f'{col}_lag_{lag}'] = pivot[col].shift(lag)
pivot.dropna()

Type,humidity,pm10,pm25,temperature,humidity_lag_1,pm10_lag_1,pm25_lag_1,temperature_lag_1,humidity_lag_2,pm10_lag_2,pm25_lag_2,temperature_lag_2,humidity_lag_3,pm10_lag_3,pm25_lag_3,temperature_lag_3
Stamp,,,,,,,,,,,,,,,,
2023-01-01 00:45:00+00:00,87.0,415.0,125.0,5.0,87.0,328.0,108.0,5.0,87.0,333.0,120.0,5.0,87.0,473.0,156.0,5.0
2023-01-01 01:00:00+00:00,87.0,341.0,110.0,5.0,87.0,415.0,125.0,5.0,87.0,328.0,108.0,5.0,87.0,333.0,120.0,5.0
2023-01-01 01:15:00+00:00,87.0,326.0,110.0,5.0,87.0,341.0,110.0,5.0,87.0,415.0,125.0,5.0,87.0,328.0,108.0,5.0
2023-01-01 01:30:00+00:00,87.0,330.0,110.0,5.0,87.0,326.0,110.0,5.0,87.0,341.0,110.0,5.0,87.0,415.0,125.0,5.0
2023-01-01 01:45:00+00:00,87.0,337.0,112.0,5.0,87.0,330.0,110.0,5.0,87.0,326.0,110.0,5.0,87.0,341.0,110.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-11-21 21:45:00+00:00,74.0,108.0,47.0,2.0,74.0,107.0,49.0,2.0,73.0,115.0,52.0,3.0,71.0,119.0,49.0,3.0
2024-11-21 22:00:00+00:00,74.0,79.0,35.0,2.0,74.0,108.0,47.0,2.0,74.0,107.0,49.0,2.0,73.0,115.0,52.0,3.0
2024-11-21 22:15:00+00:00,74.0,81.0,33.0,2.0,74.0,79.0,35.0,2.0,74.0,108.0,47.0,2.0,74.0,107.0,49.0,2.0


## Check Correlation between the extracted features and target

In [15]:
# Write your code here. Add as many boxes as you need.
pivot.corr()

Type,humidity,pm10,pm25,temperature,humidity_lag_1,pm10_lag_1,pm25_lag_1,temperature_lag_1,humidity_lag_2,pm10_lag_2,pm25_lag_2,temperature_lag_2,humidity_lag_3,pm10_lag_3,pm25_lag_3,temperature_lag_3
Type,,,,,,,,,,,,,,,,
humidity,1.000000,0.255162,0.254214,-0.648651,0.997380,0.255920,0.254852,-0.647524,0.992945,0.256284,0.255096,-0.645521,0.987116,0.256440,0.255135,-0.642849
pm10,0.255162,1.000000,0.976441,-0.240952,0.254049,0.979657,0.965269,-0.240556,0.253000,0.971983,0.960323,-0.240148,0.251874,0.967354,0.956631,-0.239701
pm25,0.254214,0.976441,1.000000,-0.230558,0.253198,0.966452,0.989009,-0.230201,0.252162,0.961090,0.984429,-0.229842,0.251071,0.956351,0.980707,-0.229418
temperature,-0.648651,-0.240952,-0.230558,1.000000,-0.648594,-0.241248,-0.230782,0.998939,-0.647309,-0.241434,-0.230905,0.997660,-0.645026,-0.241528,-0.230947,0.996073
humidity_lag_1,0.997380,0.254049,0.253198,-0.648594,1.000000,0.255165,0.254217,-0.648645,0.997380,0.255921,0.254853,-0.647518,0.992945,0.256284,0.255097,-0.645514
pm10_lag_1,0.255920,0.979657,0.966452,-0.241248,0.255165,1.000000,0.976441,-0.240958,0.254052,0.979657,0.965269,-0.240562,0.253002,0.971983,0.960323,-0.240154
pm25_lag_1,0.254852,0.965269,0.989009,-0.230782,0.254217,0.976441,1.000000,-0.230565,0.253201,0.966452,0.989009,-0.230208,0.252165,0.961090,0.984429,-0.229849
temperature_lag_1,-0.647524,-0.240556,-0.230201,0.998939,-0.648645,-0.240958,-0.230565,1.000000,-0.648588,-0.241251,-0.230786,0.998939,-0.647302,-0.241435,-0.230909,0.997660
humidity_lag_2,0.992945,0.253000,0.252162,-0.647309,0.997380,0.254052,0.253201,-0.648588,1.000000,0.255166,0.254218,-0.648638,0.997380,0.255921,0.254854,-0.647511


---
# Train and test models

## Dataset Splitting
Partition the dataset into training and testing sets with an 80:20 ratio.

**RECOMMENDATION: Use only first 1 000 data points.**

**WARNING: DO NOT SHUFFLE THE DATASET.**

**Columns for prediction:**
- 'humidity'
- 'pm25'
- 'pm10' 
- 'temperature'

In [16]:
pivot.sort_index(inplace=True)

In [17]:
# Write your code here. Add as many boxes as you need.
pivot = pivot.head(1000)

In [18]:
# Write your code here. Add as many boxes as you need.
X = pivot.drop(columns=['humidity', 'pm25', 'pm10', 'temperature'])
y = pivot[['humidity', 'pm25', 'pm10', 'temperature']]

In [19]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

## Ensemble Learning Methods

### Bagging

Create an instance of a Random Forest model and train it using the `fit` function.

In [22]:
# Write your code here. Add as many boxes as you need.
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

rf_models = {col: RandomForestRegressor(n_estimators=100, random_state=42) for col in y.columns}

for col in y.columns:
    rf_models[col].fit(X_train, y_train[col])

Use the trained model to make predictions for the test set.

Assess the performance of the model by using different metrics provided by the `scikit-learn` library.

Output independent metrics for each column you predict. 

In [23]:
# Write your code here. Add as many boxes as you need.
for col in y.columns:
    y_pred = rf_models[col].predict(X_test)
    print(f"\nRandom Forest Metrics for {col}:")
    print(f"MAE: {mean_absolute_error(y_test[col], y_pred):.4f}")
    print(f"MSE: {mean_squared_error(y_test[col], y_pred):.4f}")
    print(f"R2: {r2_score(y_test[col], y_pred):.4f}")


Random Forest Metrics for humidity:
MAE: 0.6758
MSE: 0.6600
R2: 0.9650

Random Forest Metrics for pm25:
MAE: 18.6771
MSE: 603.3860
R2: 0.4174

Random Forest Metrics for pm10:
MAE: 34.0648
MSE: 1817.7425
R2: 0.4989

Random Forest Metrics for temperature:
MAE: 0.2679
MSE: 0.1798
R2: 0.9410


### Boosting

Create an instance of an XGBoost model and train it using the `fit` function.

In [24]:
# Write your code here. Add as many boxes as you need.
from xgboost import XGBRegressor

xgb_models = {col: XGBRegressor(n_estimators=100,random_state=42) for col in y.columns}

for col in y.columns:
    xgb_models[col].fit(X_train, y_train[col])

Use the trained model to make predictions for the test set.

Assess the performance of the model by using different metrics provided by the `scikit-learn` library.

Output independent metrics for each column you predict. 

In [25]:
# Write your code here. Add as many boxes as you need.
for col in y.columns:
    y_pred = xgb_models[col].predict(X_test)
    print(f"\nXGBoost Metrics for {col}:")
    print(f"MAE: {mean_absolute_error(y_test[col], y_pred):.4f}")
    print(f"MSE: {mean_squared_error(y_test[col], y_pred):.4f}")
    print(f"R2: {r2_score(y_test[col], y_pred):.4f}")


XGBoost Metrics for humidity:
MAE: 0.7009
MSE: 0.6917
R2: 0.9633

XGBoost Metrics for pm25:
MAE: 19.0091
MSE: 541.4088
R2: 0.4773

XGBoost Metrics for pm10:
MAE: 37.8247
MSE: 2054.9653
R2: 0.4335

XGBoost Metrics for temperature:
MAE: 0.3953
MSE: 0.3023
R2: 0.9008


# Laboratory Exercise - Bonus Task (+ 2 points)

As part of the bonus task in this laboratory assignment, your objective is to fine-tune the number of estimators (`n_estimators`) for the XGBoost model using a cross-validation with grid search and time series split. This involves systematically experimenting with various values for `n_estimators` and evaluating the model's performance using cross-validation. Upon determining the most suitable `n_estimators` value, evaluate the model's performance on a test set for final assessment.

Hints:
- For grid search use the `GridCVSearch` from the `scikit-learn` library. Check the documentation at https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html.
- For cross-validation use the `TimeSeriesSplit` from the `scikit-learn` library. Check the documentation at https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.TimeSeriesSplit.html.

## Dataset Splitting
Partition the dataset into training and testing sets with an 90:10 ratio.

**WARNING: DO NOT SHUFFLE THE DATASET.**

In [26]:
# Write your code here. Add as many boxes as you need.
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, shuffle=False)

## Fine-tuning the XGBoost Hyperparameter
Experiment with various values for `n_estimators` and evaluate the model's performance using cross-validation.

In [34]:
# Write your code here. Add as many boxes as you need.
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit

tscv = TimeSeriesSplit(n_splits=5)
param_grid = {
    'n_estimators' : [50, 100, 200, 300]
}
for col in y.columns:
    grid_search = GridSearchCV(
        estimator=XGBRegressor(random_state=42),
        param_grid=param_grid,
        scoring='neg_mean_squared_error',
        cv=tscv,
        verbose=1
    )
    grid_search.fit(X_train, y_train[col])

    print(f'Best params for {col}: {grid_search.best_params_}')

Fitting 5 folds for each of 4 candidates, totalling 20 fits
Best params for humidity: {'n_estimators': 50}
Fitting 5 folds for each of 4 candidates, totalling 20 fits
Best params for pm25: {'n_estimators': 50}
Fitting 5 folds for each of 4 candidates, totalling 20 fits
Best params for pm10: {'n_estimators': 50}
Fitting 5 folds for each of 4 candidates, totalling 20 fits
Best params for temperature: {'n_estimators': 300}


## Final Assessment of the Model Performance
Upon determining the most suitable `n_estimators` value, evaluate the model's performance on a test set for final assessment.

In [38]:
# Write your code here. Add as many boxes as you need.
params = [50,50,50,300]
for i,col in enumerate(y.columns):
    best_model = XGBRegressor(n_estimators=params[i])
    best_model.fit(X_train,y_train[col])
    y_pred = best_model.predict(X_test)
    print(f"\nMetrics for {col} with {best_model.n_estimators} estimators:")
    print(f"MAE: {mean_absolute_error(y_test[col], y_pred):.4f}")
    print(f"MSE: {mean_squared_error(y_test[col], y_pred):.4f}")
    print(f"R2: {r2_score(y_test[col], y_pred):.4f}")


Metrics for humidity with 50 estimators:
MAE: 0.6948
MSE: 0.7417
R2: 0.9454

Metrics for pm25 with 50 estimators:
MAE: 18.9151
MSE: 466.4952
R2: -0.9526

Metrics for pm10 with 50 estimators:
MAE: 37.6527
MSE: 1780.7578
R2: -1.2473

Metrics for temperature with 300 estimators:
MAE: 0.4970
MSE: 0.3791
R2: 0.9089
